# Biolink Metamodel Test Notebook

In [1]:
!pip -q --disable-pip-version-check install yamlmagic
%reload_ext yamlmagic

In [2]:
from IPython.core.display import display, HTML
from types import ModuleType
from json import loads, dumps
from jsonasobj import JsonObj, as_json
from rdflib import Graph

from linkml.meta import SchemaDefinition
from linkml.utils.schemaloader import SchemaLoader
from linkml.utils.yamlutils import DupCheckYamlLoader, as_json_object as yaml_to_json

from linkml.generators.shexgen import ShExGenerator
from linkml.generators.pythongen import PythonGenerator
from linkml.generators.yumlgen import YumlGenerator
from linkml.generators.jsonldcontextgen import ContextGenerator

## Basic model structure
A biolink model consists of:
* a name
* a uri
* type definitions
* slot definitions
* class definitions
* subset definitions

As an example, the model below defines:

In [3]:
%%yaml --loader DupCheckYamlLoader yaml
id: http://example.org/sample/example1
name: synopsis2
prefixes:
    foaf: http://xmlns.com/foaf/0.1/
    samp: http://example.org/model/
    xsd: http://www.w3.org/2001/XMLSchema#

default_prefix: samp

default_curi_maps:
    - semweb_context

default_range: string

types:
    string:
        base: str
        uri: xsd:string
    int:
        base: int
        uri: xsd:integer
    boolean:
        base: Bool
        uri: xsd:boolean


classes:
    person:
        description: A person, living or dead
        slots:
            - id
            - first name
            - last name
            - age
            - living
            - knows

    friendly_person:
        description: Any person that knows someone
        is_a: person
        slot_usage:
            knows:
                required: True

slots:
    id:
        description: Unique identifier of a person
        identifier: true

    first name:
        description: The first name of a person
        slot_uri: foaf:firstName
        multivalued: true

    last name:
        description: The last name of a person
        slot_uri: foaf:lastName
        required: true

    living:
        description: Whether the person is alive
        range: boolean
        comments:
            - unspecified means unknown

    age:
        description: The age of a person if living or age of death if not
        range: int
        slot_uri: foaf:age

    knows:
        description: A person known by this person (indicating some level of reciprocated interaction between the parties).
        range: person
        slot_uri: foaf:knows
        multivalued: true

<IPython.core.display.Javascript object>

### We can emit this model as a Python class

In [4]:
print(PythonGenerator(yaml).serialize())

# Auto generated from None by pythongen.py version: 0.9.0
# Generation date: 2021-01-05 15:24
# Schema: synopsis2
#
# id: http://example.org/sample/example1
# description:
# license:

import dataclasses
import sys
import re
from typing import Optional, List, Union, Dict, ClassVar, Any
from dataclasses import dataclass
from linkml.meta import EnumDefinition, PermissibleValue, PvFormulaOptions

from linkml.utils.slot import Slot
from linkml.utils.metamodelcore import empty_list, empty_dict, bnode
from linkml.utils.yamlutils import YAMLRoot, extended_str, extended_float, extended_int
if sys.version_info < (3, 7, 6):
    from linkml.utils.dataclass_extensions_375 import dataclasses_init_fn_with_kwargs
else:
    from linkml.utils.dataclass_extensions_376 import dataclasses_init_fn_with_kwargs
from linkml.utils.formatutils import camelcase, underscore, sfx
from linkml.utils.enumerations import EnumDefinitionImpl
from rdflib import Namespace, URIRef
from linkml.utils.curienamespace import Cur

### Compile the python into a module

In [5]:
spec = compile(PythonGenerator(yaml).serialize(), 'test', 'exec')
module = ModuleType('test')
exec(spec, module.__dict__)

### We can emit a UML rendering of  this model

In [6]:
display(HTML(f'<img src="{YumlGenerator(yaml).serialize()}"/>'))

### We can emit a JSON-LD context for the model:

In [7]:
cntxt = loads(ContextGenerator(yaml).serialize(base="http://example.org/people/"))
print(dumps(cntxt, indent="  "))

{
  "_comments": "Auto generated from None by jsonldcontextgen.py version: 0.1.1\nGeneration date: 2021-01-05 15:24\nSchema: synopsis2\n\nid: http://example.org/sample/example1\ndescription: \nlicense: \n",
  "@context": {
    "type": "@type",
    "foaf": "http://xmlns.com/foaf/0.1/",
    "samp": "http://example.org/model/",
    "xsd": "http://www.w3.org/2001/XMLSchema#",
    "@vocab": "http://example.org/model/",
    "age": {
      "@type": "xsd:integer",
      "@id": "foaf:age"
    },
    "first_name": {
      "@id": "foaf:firstName"
    },
    "id": "@id",
    "knows": {
      "@type": "@id",
      "@id": "foaf:knows"
    },
    "last_name": {
      "@id": "foaf:lastName"
    },
    "living": {
      "@type": "xsd:boolean"
    },
    "@base": "http://example.org/people/"
  }
}


### The python model can be used to create classes

In [8]:
# Generate a person
joe_smith = module.Person(id="42", last_name="smith", first_name=['Joe', 'Bob'], age=43)
print(joe_smith)

Person(id='42', last_name='smith', first_name=['Joe', 'Bob'], age=43, living=None, knows=[])


### and can be combined w/ the JSON-LD Context to generate RDF

In [9]:
# Add the context and turn it into RDF
jsonld = as_json(yaml_to_json(joe_smith, cntxt))
print(jsonld)
g = Graph()
g.parse(data=jsonld, format="json-ld")
print(g.serialize(format="turtle").decode())

{
   "id": "42",
   "last_name": "smith",
   "first_name": [
      "Joe",
      "Bob"
   ],
   "age": 43,
   "living": null,
   "knows": [],
   "type": "Person",
   "@context": {
      "type": "@type",
      "foaf": "http://xmlns.com/foaf/0.1/",
      "samp": "http://example.org/model/",
      "xsd": "http://www.w3.org/2001/XMLSchema#",
      "@vocab": "http://example.org/model/",
      "age": {
         "@type": "xsd:integer",
         "@id": "foaf:age"
      },
      "first_name": {
         "@id": "foaf:firstName"
      },
      "id": "@id",
      "knows": {
         "@type": "@id",
         "@id": "foaf:knows"
      },
      "last_name": {
         "@id": "foaf:lastName"
      },
      "living": {
         "@type": "xsd:boolean"
      },
      "@base": "http://example.org/people/"
   }
}
@prefix : <http://example.org/model/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix samp: <http://example.org/model/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.

### The model can be turned into ShEx

In [10]:
shex = ShExGenerator(yaml).serialize(collections=False)
print(shex)

BASE <http://example.org/model/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>


<String> xsd:string

<Int> xsd:integer

<Boolean> xsd:boolean

<FriendlyPerson> CLOSED {
    (  $<FriendlyPerson_tes> (  &<Person_tes> ;
          rdf:type [ <Person> ] ? ;
          <knows> @<Person> +
       ) ;
       rdf:type [ <FriendlyPerson> ]
    )
}

<Person>  (
    CLOSED {
       (  $<Person_tes> (  foaf:firstName @<String> * ;
             foaf:lastName @<String> ;
             foaf:age @<Int> ? ;
             <living> @<Boolean> ? ;
             foaf:knows @<Person> *
          ) ;
          rdf:type [ <Person> ]
       )
    } OR @<FriendlyPerson>
)





### The ShEx can then be used to validate RDF

In [11]:
from pyshex.evaluate import evaluate
r = evaluate(g, shex,
             start="http://example.org/model/Person",
             focus="http://example.org/people/42")
print("Conforms" if r[0] else r[1])

Conforms


In [12]:
r = evaluate(g, shex,
             start="http://example.org/model/FriendlyPerson",
             focus="http://example.org/people/42")
print("Conforms" if r[0] else r[1])

  Testing <http://example.org/people/42> against shape http://example.org/model/FriendlyPerson
       No matching triples found for predicate samp:knows
